# Robinhood API Usage Examples

In [3]:
import getpass
import requests
import numpy as np
import pandas as pd
from pyrh import Robinhood
import qgrid
import xlsxwriter
from dateutil import parser

#Make sure you install pyrh and qgrid either using conda install or pip install
#Install All the sufficient libraries in your local machine

In [4]:
username = input('Username :: ')
password = getpass.getpass('Password :: ')
print("Check your mobile for 2 factor Authentication Code:")
rh = Robinhood()
rh.login(username, password,challenge_type="sms")

Username :: arun.ap.90@gmail.com
Password :: ········
Check your mobile for 2 factor Authentication Code:
No 2FA Given
sms code:
087029


True

# CODE TO PREPARE ROBINHOOD STOCK ACCOUNT DATA MERGE WITH SECTOR AND INDUSTRY DATA

In [9]:
def get_order_History(ordertype):
    if ordertype == 'orders':
        history = rh.order_history()
    elif ordertype == 'dividends':
        history = rh.dividends()
    historyList = history['results']
    while True:
        if(history['next'] is None):
            break
        history = rh.get_url(history['next'])
        historyList.extend(history['results'])
    return historyList

#print('Total Orders in your RobinHood Account:',len(get_order_History()))

def get_dividends(dividendHistoryList):
    dividendDict ={}
    dividendList =[]
    for dividend in dividendHistoryList:
        instrumentResponse = requests.get(dividend["instrument"])
        instrumentJSON = instrumentResponse.json()
        # print(instrumentJSON)

        dividendDict['name'] = instrumentJSON["simple_name"]
        dividendDict['ticker'] = instrumentJSON["symbol"]
        dividendDict['amount'] = dividend["amount"]
        dividendDict['payable_date'] = dividend["payable_date"]
        dividendList.append(dividendDict.copy())
    #print(dividendList)
    return sorted(dividendList,key=lambda k:k['payable_date'])


def get_all_orders_sortedByDate(orderHistoryList):
    stockDict = {}
    allStockList = []
    for order in orderHistoryList:
        if(order['state'] != "cancelled" and order['state'] != "confirmed" and order['state'] != "rejected"):
            instrumentResponse = requests.get(order["instrument"])
            instrumentJSON = instrumentResponse.json()
            # print(instrumentJSON)
            stockDict['name'] = instrumentJSON["simple_name"]
            stockDict['ticker'] = instrumentJSON["symbol"]
            stockDict['date'] = parser.parse(order["executions"][0]["settlement_date"])
            stockDict['tranType'] = order["side"]
            stockDict['totalBuyingPrice'] = order["executed_notional"]["amount"]
            
            splitStock = 1
            if stockDict['ticker'] == 'AAPL' and stockDict['date'] < parser.parse('2020-09-01 00:00:00'):
                splitStock = 4
            elif stockDict['ticker'] == 'TSLA' and stockDict['date'] < parser.parse('2020-08-30 00:00:00') : 
                splitStock = 5
            
            stockDict['BuyingPricePerShare'] = float(order["average_price"])/splitStock
            stockDict['quantity'] = float(order["cumulative_quantity"])*splitStock
            
            allStockList.append(stockDict.copy())
    return sorted(allStockList,key=lambda k:k['date'])

def get_all_orders_FilterByDate(orderHistoryList,filterDate):
    stockDict = {}
    allStockList = []
    for order in orderHistoryList:
        if(order['state'] != "cancelled" and order['state'] != "confirmed" and order['state'] != "rejected") and parser.parse(order["executions"][0]["settlement_date"]) >= parser.parse(filterDate):
            instrumentResponse = requests.get(order["instrument"])
            instrumentJSON = instrumentResponse.json()
            # print(instrumentJSON)
            stockDict['name'] = instrumentJSON["simple_name"]
            stockDict['ticker'] = instrumentJSON["symbol"]
            stockDict['date'] = parser.parse(order["executions"][0]["settlement_date"])
            stockDict['tranType'] = order["side"]
            stockDict['totalBuyingPrice'] = order["executed_notional"]["amount"]
            
            splitStock = 1
            if stockDict['ticker'] == 'AAPL' and stockDict['date'] < parser.parse('2020-09-01 00:00:00'):
                splitStock = 4
            elif stockDict['ticker'] == 'TSLA' and stockDict['date'] < parser.parse('2020-08-30 00:00:00') : 
                splitStock = 5
            
            stockDict['BuyingPricePerShare'] = float(order["average_price"])/splitStock
            stockDict['quantity'] = float(order["cumulative_quantity"])*splitStock
            
            allStockList.append(stockDict.copy())
    return sorted(allStockList,key=lambda k:k['date'])

def split_BuyAndSellList(sortedOrders):
    buyList = []
    sellList = []
    tickerList =[]
    stockMap ={}
    for row in sortedOrders:
        if row['tranType'] == 'buy':
            if row['ticker'] not in stockMap:
                stockMap[row['ticker']] = row
                tickerList.append(row['ticker'])
            else:
                existingRow = stockMap[row['ticker']]
                newRow ={}
                newRow['name'] = str(row['name'])
                newRow['ticker'] = str(row['ticker'])
                newRow['tranType'] = str(row['tranType'])
                newRow['date'] = str(row['date'])
                newRow['quantity'] = float(row['quantity']) + float(existingRow['quantity'])
                newRow['totalBuyingPrice'] = float(row['totalBuyingPrice']) + float(existingRow['totalBuyingPrice'])
                newRow['BuyingPricePerShare'] = float(newRow['totalBuyingPrice'])/float(newRow['quantity'])
                stockMap[row['ticker']] = newRow
        else:
            if(row['ticker'] in stockMap):
                alreadyBoughtRow = stockMap.pop(row['ticker'])
                if(alreadyBoughtRow['ticker'] == row['ticker']):   
                    if((float(alreadyBoughtRow['quantity']) - float(row['quantity'])) != 0.0):
                        alreadyBoughtRow['quantity'] = float(alreadyBoughtRow['quantity']) - float(row['quantity'])
                        alreadyBoughtRow['totalBuyingPrice'] = float(alreadyBoughtRow['BuyingPricePerShare'])*float(alreadyBoughtRow['quantity'])
                        stockMap[row['ticker']] = alreadyBoughtRow
                    soldRow = alreadyBoughtRow.copy()
                    soldRow['SoldDate'] = str(row['date'])
                    soldRow['tranType'] = 'Sold'
                    soldRow['sellingQuantity'] = float(row['quantity'])
                    soldRow['totalSellingPrice'] = float(row['totalBuyingPrice'])
                    soldRow['SellingPricePerShare'] = float(row['BuyingPricePerShare'])
                    sellList.append(soldRow)
    buyList = stockMap.values()
    return buyList,sellList,tickerList

def get_curr_marketPrice(tickerList):
    currPriceList = rh.quotes_data(set(tickerList))
    #currPriceList
    currDict = {}
    currStockList = []
    for curr in currPriceList:
        if curr is not None:
            #if curr['symbol']:
            #    currDict['ticker'] = curr['symbol']
            #    currDict['currPrice'] = curr['symbol' == 'CZR']['last_trade_price']
            #else:
            currDict['ticker'] = curr['symbol']
            currDict['currPrice'] = curr['last_trade_price']
            currStockList.append(currDict.copy())
    return currStockList

def fillnaByTypes(df):
    for col in df:
        #get dtype for column
        dt = df[col].dtype 
        #check if it is a number
        if dt == int or dt == float:
            df[col].fillna(0)
        else:
            df[col].fillna("Unknown")
    return df;

def prepare_data_frames(buyList,currPriceList):
    df1 = pd.DataFrame(buyList)
    convert_type = {'name' : str,
                    'ticker' : str,
                    'BuyingPricePerShare' : float,
                    'totalBuyingPrice' : float,
                    'quantity' : float,
                    'date' : str,
                    'tranType' : str
                   }
    df1 = df1.astype(convert_type)
    df1 = df1.rename(columns={  'name' : 'Name',
                                'ticker' : 'Ticker',
                                'BuyingPricePerShare' : 'Purchase Price (Per Share)',
                                'totalBuyingPrice' : 'Total Purchase Price',
                                'quantity' : 'Purchase Quantity',
                                'date' : 'Purchase Date',
                                'tranType' : 'Transaction Type'
                   })
    #print(df1.dtypes)

    df2 = pd.DataFrame(currPriceList)
    convert_type = {'ticker' : str,
                    'currPrice' : float
                   }
    df2 = df2.astype(convert_type)
    df2 = df2.rename(columns={  'ticker' : 'Ticker',
                                'currPrice' : 'Current Market Price (Per Share)'
                   })
    #print(df2.dtypes)
    
    df3 = pd.read_csv('companylist.csv',usecols=['Symbol','Sector','industry'])
    df3.columns = ['Ticker','Sector','Industry']
    convert_type = {'Ticker' : str,
                    'Sector' : str,
                    'Industry' : str
                   }
    
    df3 = df3.astype(convert_type)
    df3 = df3.drop_duplicates()
    #print(df3.dtypes)
    
    portfolio = df1.merge(df2,on='Ticker',how='left').merge(df3,on='Ticker',how='left')
    portfolio = portfolio.sort_values('Purchase Date',ascending=False)
    portfolio.reset_index(inplace=True,drop=True)
    portfolio = fillnaByTypes(portfolio)
    return portfolio

def re_order_columns(portfolio):
    return portfolio[['Name',
                     'Ticker',
                     'Purchase Date',
                     'Transaction Type',
                     'Purchase Quantity',
                     'Purchase Price (Per Share)',
                     'Current Market Price (Per Share)',
                     'Total Purchase Price',
                     'Total Market Price',
                     'Sector',
                     'Industry',
                     'Profit/Loss',
                     'Profit/Loss Percentage',
                     'Portfolio Diversity'
                     ]].sort_values('Profit/Loss Percentage')
'''
def create_calculated_fields(portfolio):
    portfolio['Total Market Price'] = round(portfolio['Current Market Price (Per Share)'] * portfolio['Purchase Quantity'],2)
    portfolio["Profit/Loss"] = round(portfolio["Total Market Price"] - portfolio["Total Purchase Price"],2)
    portfolio["Profit/Loss Percentage"] = round((portfolio["Profit/Loss"]/portfolio["Total Purchase Price"])*100,2)
    portfolio["Portfolio Diversity"] = round((portfolio['Total Market Price']/portfolio['Total Market Price'].sum())*100,2)

    #portfolio["Profit/Loss Percentage"] = portfolio["Profit/Loss Percentage"].astype(str)
    #portfolio["Profit/Loss Percentage"] = portfolio["Profit/Loss Percentage"]  + "%"
    #portfolio["Portfolio Diversity"] = portfolio["Portfolio Diversity"].astype(str)
    #portfolio["Portfolio Diversity"] = portfolio["Portfolio Diversity"]  + "%"

    portfolio["Purchase Price (Per Share)"] = portfolio["Purchase Price (Per Share)"].round(2)
    portfolio["Total Market Price"] = portfolio["Total Market Price"].round(2)
    return re_order_columns(portfolio)
'''


def create_calculated_fields(portfolio):
    portfolio['Total Market Price'] = np.round(portfolio['Current Market Price (Per Share)'] * portfolio['Purchase Quantity'],2)
    portfolio["Profit/Loss"] = np.round(portfolio["Total Market Price"] - portfolio["Total Purchase Price"],2)
    portfolio["Profit/Loss Percentage"] = np.round((portfolio["Profit/Loss"]/portfolio["Total Purchase Price"])*100,2)
    portfolio["Portfolio Diversity"] = np.round((portfolio['Total Market Price']/portfolio['Total Market Price'].sum())*100,2)

    #portfolio["Profit/Loss Percentage"] = portfolio["Profit/Loss Percentage"].astype(str)
    #portfolio["Profit/Loss Percentage"] = portfolio["Profit/Loss Percentage"]  + "%"
    #portfolio["Portfolio Diversity"] = portfolio["Portfolio Diversity"].astype(str)
    #portfolio["Portfolio Diversity"] = portfolio["Portfolio Diversity"]  + "%"

    portfolio["Purchase Price (Per Share)"] = portfolio["Purchase Price (Per Share)"].round(2)
    portfolio["Total Market Price"] = portfolio["Total Market Price"].round(2)
    return re_order_columns(portfolio)

def get_Sector_and_Industry_Analysis(finalSummary):
    withIndustryColumns = [
                     'Sector',
                     'Industry',
                     'Total Purchase Price',
                     'Total Market Price',
                     'Profit/Loss'
                     ]
    #portfolioAnalysisWithIndustry = finalSummary[withIndustryColumns].groupby(['Industry'],as_index=False)
    #portfolioAnalysisWithSector = finalSummary[withIndustryColumns].groupby(['Sector'],as_index=False)
    industryAnalysis = finalSummary[withIndustryColumns].groupby(['Industry'],as_index=False).agg({'Total Purchase Price': np.sum,'Total Market Price': np.sum,'Profit/Loss': np.sum})  
    sectorAnalysis = finalSummary[withIndustryColumns].groupby(['Sector'],as_index=False).agg({'Total Purchase Price': np.sum,'Total Market Price': np.sum,'Profit/Loss': np.sum})  
    sectorAnalysis["Profit/Loss Percentage"] = round((sectorAnalysis["Profit/Loss"]/sectorAnalysis["Total Purchase Price"])*100,2)
    sectorAnalysis["Portfolio Diversity"] = round((sectorAnalysis['Total Market Price']/sectorAnalysis['Total Market Price'].sum())*100,2)
    industryAnalysis["Profit/Loss Percentage"] = round((industryAnalysis["Profit/Loss"]/industryAnalysis["Total Purchase Price"])*100,2)
    industryAnalysis["Portfolio Diversity"] = round((industryAnalysis['Total Market Price']/industryAnalysis['Total Market Price'].sum())*100,2)
    dropColumns =['Total Purchase Price','Total Market Price','Profit/Loss']
    sectorSummary = sectorAnalysis.drop(dropColumns,axis=1)
    industrySummary = industryAnalysis.drop(dropColumns,axis=1)
    return sectorSummary.sort_values('Profit/Loss Percentage'),industrySummary.sort_values('Profit/Loss Percentage')
    #,portfolioAnalysisWithIndustry.sort_values('Profit/Loss Percentage'),portfolioAnalysisWithSector.sort_values('Profit/Loss Percentage')

def process_data_from_robinHood(filterdate):
    orderHistoryList              =   get_order_History('orders')
    dividendHistoryList           =   get_order_History('dividends')
    dividendList                  =   get_dividends(dividendHistoryList)
    if filterdate is  None :
        sortedOrders                  =   get_all_orders_sortedByDate(orderHistoryList)
    else:
        sortedOrders                  =   get_all_orders_FilterByDate(orderHistoryList,filterdate)
    buyList,sellList,tickerList   =   split_BuyAndSellList(sortedOrders)
    currPriceList                 =   get_curr_marketPrice(tickerList)
    portfolio                     =   prepare_data_frames(buyList,currPriceList)
    finalSummary                  =   create_calculated_fields(portfolio)
    sectorSummary,industrySummary =   get_Sector_and_Industry_Analysis(finalSummary)
    return finalSummary,sectorSummary,industrySummary,sellList,dividendList

def write_to_Excel(finalSummary,sectorSummary,industrySummary):
    writer = pd.ExcelWriter('stock_Analysis_RobinHood.xlsx', engine='xlsxwriter')
    finalSummary.to_excel(writer, sheet_name='Final Summary',index=0)
    sectorSummary
    sectorSummary.to_excel(writer, sheet_name='Sector Summary',index=0)
    industrySummary.to_excel(writer, sheet_name='Industry Summary',index=0)
    workbook = writer.book
    
    formatPercentage = workbook.add_format({'num_format': '0.00\%'})
    
    sectorXlsx = writer.sheets['Sector Summary']
    sectorXlsx.set_column('B:C', None, formatPercentage)
    column_chart1 = workbook.add_chart({'type': 'column'})
    column_chart1.add_series({
    'name':     ['Sector Summary', 0, 1, 0, 1],    
    'values':     ['Sector Summary', 1, 1, sectorSummary.shape[0]-1, 1],
    'categories':     ['Sector Summary', 1, 0, sectorSummary.shape[0]-1, 0],
    })
    line_chart2 = workbook.add_chart({'type': 'line'})
    line_chart2.add_series({
    'name':     ['Sector Summary', 0, 2, 0, 2],    
    'values':     ['Sector Summary', 1, 2, sectorSummary.shape[0]-1, 2],
    'categories':     ['Sector Summary', 1, 0, sectorSummary.shape[0]-1, 0],
    })
    column_chart1.combine(line_chart2)
    column_chart1.set_title({ 'name': 'Sector Analysis of Portfolio Diversity and P/L %'})
    column_chart1.set_x_axis({'name': 'Sector','label_position': 'low'})
    column_chart1.set_y_axis({'name': 'Percentage %'})
    column_chart1.set_size({'width': 720, 'height': 576})

    sectorXlsx.insert_chart('F2', column_chart1)
    
    
    industryXlsx = writer.sheets['Industry Summary']
    industryXlsx.set_column('B:C', None, formatPercentage)
    column_chart1 = workbook.add_chart({'type': 'bar'})
    column_chart1.add_series({
    'name':     ['Industry Summary', 0, 1, 0, 1],    
    'values':     ['Industry Summary', 1, 1, industrySummary.shape[0]-1, 1],
    'categories':     ['Industry Summary', 1, 0, industrySummary.shape[0]-1, 0],
    })
    #line_chart2 = workbook.add_chart({'type': 'line'})
    #line_chart2.add_series({
    #'name':     ['Industry Summary', 0, 2, 0, 2],    
    #'values':     ['Industry Summary', 1, 2, industrySummary.shape[0]-1, 2],
    #'categories':     ['Industry Summary', 1, 0, industrySummary.shape[0]-1, 0],
    #})
    #column_chart1.combine(line_chart2)
    column_chart1.set_title({ 'name': 'Industry Analysis of Portfolio Diversity and P/L %'})
    column_chart1.set_x_axis({'name': 'Percentage %','label_position': 'low'})
    column_chart1.set_y_axis({'name': 'Industry','label_position': 'low'})
    column_chart1.set_size({'width': 1024, 'height': 768})

    industryXlsx.insert_chart('F2', column_chart1)
    
    writer.save()

In [30]:
finalSummary,sectorSummary,industrySummary,sellList,dividendList = process_data_from_robinHood(None)
write_to_Excel(finalSummary,sectorSummary,industrySummary)

#You can Also filter the final summary data by passing the starting date to current date. Something like below example
#finalSummary,sectorSummary,industrySummary,sellList,dividendList = process_data_from_robinHood("2020-09-01 00:00:00")
#write_to_Excel(finalSummary,sectorSummary,industrySummary)

In [31]:
withoutIndustryColumns = ['Name',
                     'Ticker',
                     'Sector',
                     'Industry',
                     'Purchase Quantity',
                     'Purchase Price (Per Share)',
                     'Current Market Price (Per Share)',
                     'Total Purchase Price',
                     'Total Market Price',
                     'Profit/Loss',
                     'Profit/Loss Percentage',
                     'Portfolio Diversity'
                     ]
withIndustryColumns = [
                     'Sector',
                     'Industry',
                     'Total Purchase Price',
                     'Total Market Price',
                     'Profit/Loss'
                     ]
MajorColumns = [     'Name',
                     'Ticker',
                     'Sector',
                     'Industry',
                     'Profit/Loss Percentage',
                     'Portfolio Diversity'
                     ]

In [45]:
profitOrders = finalSummary[withoutIndustryColumns][finalSummary['Profit/Loss Percentage'] > 0]
lossOrders   = finalSummary[withoutIndustryColumns][finalSummary['Profit/Loss Percentage'] <= 0]

profitPortfolio = round(profitOrders["Portfolio Diversity"].sum(),2)
lossPortfolio = round(lossOrders["Portfolio Diversity"].sum(),2)

marketPriceProfitOrders = profitOrders['Total Market Price'].sum()
purchasePriceProfitOrders = profitOrders['Total Purchase Price'].sum()
marketPriceLossOrders = lossOrders['Total Market Price'].sum()
purchasePriceLossOrders = lossOrders['Total Purchase Price'].sum()

plPositive = round((marketPriceProfitOrders - purchasePriceProfitOrders)/purchasePriceProfitOrders * 100,2)
plNegative = round((marketPriceLossOrders - purchasePriceLossOrders)/purchasePriceLossOrders * 100,2)
print(profitPortfolio,"% of your portfolio have {}% P/L percentage on Profit Orders :".format(plPositive))
print(lossPortfolio,"% of your portfolio have {}% P/L percentage on Profit Orders :".format(plNegative))
print("Over All Profit/Loss % in your Account:",((profitPortfolio*plPositive) + (lossPortfolio*plNegative))/100)

91.4 % of your portfolio have 26.88% P/L percentage on Profit Orders :
8.56 % of your portfolio have -10.81% P/L percentage on Profit Orders :
Over All Profit/Loss in your Account: 23.642984


In [46]:
portfolioAnalysis = qgrid.show_grid(finalSummary[MajorColumns])
portfolioAnalysis
#Full Portfolio Analysis. You can sort the below table by clicking on any of the columns

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [48]:
PrettyTableDisplay = qgrid.show_grid(finalSummary[withoutIndustryColumns])
PrettyTableDisplay
#Extensive Table of your portfolio. You can sort the below table by clicking on any of the columns

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [52]:
#print("Total Amount you invested in Stocks",finalSummary['Total Purchase Price'].sum())
#UnComment the above code block to see the total amount you invested in Robinhood without considering the profit/Loss you made in it

In [51]:
sectorSummaryDisplay = qgrid.show_grid(sectorSummary)
sectorSummaryDisplay

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [38]:
industrySummaryDisplay = qgrid.show_grid(industrySummary)
industrySummaryDisplay

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [39]:
#from plotly import plotly
import plotly.graph_objs as go
#import plotly.graph_objects as go
from plotly.offline import plot

#Code to Display Industry Analysis over Portfolio Diversity and Profit/Loss %

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=industrySummary['Industry'],
        y=industrySummary['Portfolio Diversity'],
        name='Portfolio Diversity %'
    ))

fig.add_trace(
    go.Bar(
        
        x=industrySummary['Industry'],
        y=industrySummary['Profit/Loss Percentage'],
        name='P/L %'
    ))
fig.layout.update(
    title={
        'text': "Industry Analysis over Portfolio Diversity and P/L%",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()


In [40]:
#Code to Display Sector Analysis over Portfolio Diversity and Profit/Loss %


fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=sectorSummary['Sector'],
        y=sectorSummary['Portfolio Diversity'],
        name='Portfolio Diversity %'
    ))

fig.add_trace(
    go.Bar(
        
        x=sectorSummary['Sector'],
        y=sectorSummary['Profit/Loss Percentage'],
        name='P/L %'
    ))

fig.layout.update(
    title={
        'text': "Sector Analysis over Portfolio Diversity and P/L%",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()

In [53]:
#Code to Display Industry Analysis sorted by Profit/Loss %
import plotly.express as px

bins = [-np.inf, -40, -20, 0, 20, 40, np.inf]
labels = ['<40%', '<20%', '<0%', '>0%', '>20%', '>40%']
#bins = pd.IntervalIndex.from_tuples([(-60, -40), (-20, 0), (0, 20)])
colors = {'<40%' : 'darkred', 
          '<20%' : 'red',
          '<0%'   : 'orangered',
          '>0%'  : 'lightgreen', 
          '>20%'  : 'green', 
          '>40%'  : 'darkgreen'
        }


industrySummary['label'] = pd.cut(industrySummary['Profit/Loss Percentage'],bins=bins,labels=labels)

bars = []
for label, label_df in industrySummary.groupby('label'):
    bars.append(go.Bar(x=label_df['Profit/Loss Percentage'],
                       y=label_df['Industry'],
                       name=label,
                       marker={'color': colors[label]},
                      orientation='h'))

fig = go.FigureWidget(data=bars)


fig.update_layout(
    autosize=False,
    width=1024,
    height=768,
    title={
        'text': "Industry Analysis sorted by Profit and Loss %",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


fig.show()


# ANALYSIS ON STOCKS THAT ARE SOLD

In [55]:
soldStocks = pd.DataFrame(sellList)
soldStocks = soldStocks.astype({'totalBuyingPrice': float})
soldStocks = soldStocks.astype({'sellingQuantity': float})
soldStocks = soldStocks.astype({'BuyingPricePerShare': float})
soldStocks['totalBuyingPricePerSellingQuantity'] = soldStocks['sellingQuantity'] * soldStocks['BuyingPricePerShare']
soldStocks['P/L %'] = ((soldStocks['totalSellingPrice']) - (soldStocks['totalBuyingPricePerSellingQuantity']))/(soldStocks['totalBuyingPricePerSellingQuantity'])*100

In [56]:
soldStocksDisplay = qgrid.show_grid(soldStocks)
soldStocksDisplay

#Table view of each stocks sold sorted on their Profit and Loss % analysis

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [57]:
#soldStocks[soldStocks['date'].str.match('2020')].sort_values('totalSellingPrice',ascending=False)

In [58]:
#sellPrice = soldStocks[soldStocks['date'].str.match('2020')]['totalSellingPrice'].sum() 
#buyPrice = soldStocks[soldStocks['date'].str.match('2020')]['totalBuyingPricePerSellingQuantity'].sum()

#profitOrLoss = (sellPrice - buyPrice)/buyPrice *100
#print('Total Selling Price : ',  sellPrice)
#print('Total Buying Price  : ' ,  buyPrice)
#print('Total P/L           : ' ,  round(profitOrLoss,2),'%')



# ANALYSIS ON DIVIDENDS EARNED

In [59]:
dividends = pd.DataFrame(dividendList)
dividends = dividends.astype({'amount': float})
print("Total Dividends Earned till date:",dividends['amount'].sum())
dividends.groupby(['name','ticker'])['amount'].sum()

Total Dividends Earned till date: 119.25


name                                                      ticker
AG Mortgage Investment Trust                              MITT       0.60
ARMOUR Residential REIT                                   ARR        4.96
Anheuser Busch InBev                                      BUD        0.34
Apple                                                     AAPL       5.76
Applied Materials                                         AMAT       1.32
Best Buy                                                  BBY        1.10
Broadcom                                                  AVGO      17.01
CVS                                                       CVS        2.01
Chevron                                                   CVX        9.03
Cisco                                                     CSCO       1.09
Darden Restaurants                                        DRI        2.68
Direxion Daily S&P Oil & Gas Exp. & Prod. Bull 2X Shares  GUSH       0.02
GE                                             

In [28]:
dividends.sort_values(['payable_date'])

,name,ticker,amount,payable_date
0,Microsoft,MSFT,1.02,2020-03-12
1,Apple,AAPL,1.64,2020-05-14
2,Visa,V,0.30,2020-06-02
3,Vanguard Intermediate-Term Corporate Bond ETF,VCIT,0.23,2020-06-04
4,Microsoft,MSFT,1.53,2020-06-11
...,...,...,...,...
71,AG Mortgage Investment Trust,MITT,0.60,2021-01-29
72,SPDR S&P 500 ETF,SPY,5.18,2021-01-29
73,Two Harbors Investment,TWO,1.79,2021-01-29
74,Darden Restaurants,DRI,1.48,2021-02-01
